In [12]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.ERROR)


def cnn_model_fn(features, labels, mode):
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])
  
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

 
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

 
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

  
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      "classes": tf.argmax(input=logits, axis=1),
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"]),
  }
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


In [13]:
# Load training and eval data
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images  # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images  # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
acc_track = []
temprature = 10
loss_weight = 0.2

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


In [14]:
writef=open('output.txt','w')

In [15]:
def current_loss(classifier,flag):
    train_eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      shuffle=False)
    eval_result1=classifier.evaluate(input_fn=train_eval_input_fn)
    if flag:
        print("\n%%%%%%% Train accuracy %%%%%%%%%%%% ",eval_result1)
        writef.write("\n%%%%%%% Train accuracy %%%%%%%%%%%% "+str(eval_result1))
    [a,b,c] = eval_result1.items()

    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": eval_data},
      y=eval_labels,
      num_epochs=1,
      shuffle=False)
    eval_result2=classifier.evaluate(input_fn=eval_input_fn)
    if flag:
        print("\n######### Test accuracy ############# ",eval_result2)
        writef.write("\n######### Test accuracy ############# "+str(eval_result2))
    [d,e,f] = eval_result2.items()
    acc_track.append([a[1],d[1]])
    return a[1],d[1]

In [16]:
# Evaluate the model and print results
mnist_classifier = tf.estimator.Estimator(
  model_fn=cnn_model_fn, model_dir="./models/mnist_convnet_model_test1")
# Set up logging for predictions
# Log the values in the "Softmax" tensor with label "probabilities"
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=50)

In [17]:
#cross entropy between two logits with temperature applied
#returns a 1d array with loss for each row in the inputs
def cross_entropy2(tar_soft_t, pred_logits_t):
   pred_soft = tf.nn.softmax(pred_logits_t)
   pred_log = tf.log(pred_soft)
   product = -1 * tar_soft_t * pred_log
   return tf.reduce_mean(product)
    
def custom_loss(y_true, pred_logits, tar_soft_t, pred_logits_t):
   return loss_weight * tf.losses.sparse_softmax_cross_entropy(labels = y_true, logits = pred_logits) + (1-loss_weight) * cross_entropy2(tar_soft_t, pred_logits_t)

In [18]:
def ensemble_model_fn(features, labels, mode):
  print(C,F,K,M,FC)
  sess=tf.InteractiveSession()
  tf.train.start_queue_runners(sess)
  data=features['x'].eval()
  out=labels.eval()
  sess.close()
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])
  eval_teacher_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x":data},
      y=out,
      num_epochs=1,
      shuffle=False)

  predictions=mnist_classifier.predict(input_fn=eval_teacher_fn)
  teacher_pred=list(predictions)
  teacher_soft=[ p['probabilities'] for p in teacher_pred]
  temp=np.array(teacher_soft)
  teacher_soft_t=tf.convert_to_tensor(temp)
#   print(teacher_soft_t)
#   for i in predictions:
#     print(i)
#   output=list(predictions)
  in_c=input_layer
  j=0
  size=28
  while j<C:
    conv = tf.layers.conv2d(
      inputs=in_c,
      filters=M,
      kernel_size=[K, K],
      padding="same",
      activation=tf.nn.relu)
    in_c = tf.layers.max_pooling2d(inputs=conv, pool_size=[2, 2], strides=2)
    size=int(size/2)
#     print(size)
    j += 1;
    


  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 7, 7, 64]
  # Output Tensor Shape: [batch_size, 7 * 7 * 64]
  flat = tf.reshape(in_c, [-1, size*size*M])

  # Dense Layer
  # Densely connected layer with 1024 neurons
  # Input Tensor Shape: [batch_size, 7 * 7 * 64]
  # Output Tensor Shape: [batch_size, 1024]
  
  j=0
  while j<F:
    flat = tf.layers.dense(inputs=flat, units=FC, activation=tf.nn.relu)
    j += 1

  # Add dropout operation; 0.6 probability that element will be kept
  dropout = tf.layers.dropout(
      inputs=flat, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits layer
  # Input Tensor Shape: [batch_size, 1024]
  # Output Tensor Shape: [batch_size, 10]
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  #   loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
  
    
#   student_pred=predictions['probabilities']
#   sess1=tf.InteractiveSession()
#   tf.train.start_queue_runners(sess1)
#   student_soft=student_pred.eval()
#   sess1.close()  
  if mode == tf.estimator.ModeKeys.TRAIN:
    pred_logits_t=tf.nn.softmax(logits/temprature, name="softmax_tensor")
#     print('a',labels,'b', logits,'c', teacher_soft_t ,'d', pred_logits_t)
    loss=custom_loss(labels, logits,teacher_soft_t , pred_logits_t)
  else:
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
  
    
  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"]),
  }
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)



In [19]:
def cal_para(C,F,K,M,FC):
    total  = C*(K*K*M);
    size=28;
    while(C):
        size=size/2
        C = C-1
    if(F==0):
        total += (size*size*M)*10
        return total;
    else:
        t=(size*size*M)
        while(F):
            total += t*FC;
            F=F-1
        total += FC*10;
        return total
    
def param_check(single_student_params,each_student_limit,each_student_params):
    global total_param
    soft_threshold = 30
    if single_student_params - total_param + soft_threshold <0:  #stop training
        return "STOP"
    else:
        if each_student_params > each_student_limit: #resample
            return "RESAMPLE"
        else :
            total_param += each_student_params
            return total_param

In [20]:
def train(C,F,K,M,FC):
    student_classifier = tf.estimator.Estimator(
      model_fn=ensemble_model_fn, model_dir="./models/ensemble/mnist_student"+"_C"+str(C)+"_F"+str(F)+"_K"+str(K)+"_M"+str(M)+"_FC"+str(FC))

#     temprature=10
#     loss_weight=0.2

    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=100,
      num_epochs=None,
      shuffle=True)

    student_classifier.train(
      input_fn=train_input_fn,
      steps=10000,
      hooks=None)
    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": eval_data},
      y=eval_labels,
      num_epochs=1,
      shuffle=False)
    eval_result=student_classifier.evaluate(input_fn=eval_input_fn)
    # output=list(predictions)
    current_loss(student_classifier,1)

In [21]:
conv=[1,2,3]
fcl=[0,1,2] 
flag=0
total_param=0
single_student_para=cal_para(2,1,5,8,256)
each_student_limit=single_student_para/20

print(single_student_para,each_student_limit)
while True:
    C=np.random.choice(conv,replace=True)
    F=np.random.choice(fcl,replace=True)
    if C+F==3:
        filters=[1,3,5,7]
        feature_map=[2,4,6,8]
        fully=[10,32,64,128]
        K=np.random.choice(filters,replace=True)
        M=np.random.choice(feature_map,replace=True)
        FC=np.random.choice(fully,replace=True)
        single_ensemble=cal_para(C,F,K,M,FC)
        choice=param_check(single_student_para,each_student_limit,single_ensemble)
        if choice=="STOP":
            print("C1")
            flag=1
            break;
        elif choice=="RESAMPLE":
            print("C2")
            continue;
        else:
            print("C3")
            s=str("\n")+str(C)+" "+str(F)+" "+str(K)+" "+str(M)+" "+str(FC)+"\n"
            writef.write(s)
            train(C,F,K,M,FC)
    if flag==1:
        break


103312.0 5165.6
C3
3 0 5 6 32
3 0 5 6 32
3 0 5 6 32
ERROR:tensorflow:Exception in QueueRunner: Attempted to use a closed Session.


Exception in thread Thread-109:
Traceback (most recent call last):
  File "/home/uttaran/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/uttaran/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/uttaran/anaconda3/lib/python3.6/site-packages/tensorflow/python/estimator/inputs/queues/feeding_queue_runner.py", line 94, in _run
    sess.run(enqueue_op, feed_dict=feed_dict)
  File "/home/uttaran/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 889, in run
    run_metadata_ptr)
  File "/home/uttaran/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1047, in _run
    raise RuntimeError('Attempted to use a closed Session.')
RuntimeError: Attempted to use a closed Session.




%%%%%%% Train accuracy %%%%%%%%%%%%  {'accuracy': 0.19254431, 'loss': 2.2634115, 'global_step': 10000}
3 0 5 6 32
ERROR:tensorflow:Exception in QueueRunner: Attempted to use a closed Session.


Exception in thread Thread-114:
Traceback (most recent call last):
  File "/home/uttaran/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/uttaran/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/uttaran/anaconda3/lib/python3.6/site-packages/tensorflow/python/estimator/inputs/queues/feeding_queue_runner.py", line 94, in _run
    sess.run(enqueue_op, feed_dict=feed_dict)
  File "/home/uttaran/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 889, in run
    run_metadata_ptr)
  File "/home/uttaran/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1047, in _run
    raise RuntimeError('Attempted to use a closed Session.')
RuntimeError: Attempted to use a closed Session.




######### Test accuracy #############  {'accuracy': 0.18781199, 'loss': 2.2626162, 'global_step': 10000}
C2
C3
3 0 1 4 10
3 0 1 4 10
ERROR:tensorflow:Exception in QueueRunner: Attempted to use a closed Session.


...

2 1 7 4 10
ERROR:tensorflow:Exception in QueueRunner: Attempted to use a closed Session.


Exception in thread Thread-1841:
Traceback (most recent call last):
  File "/home/uttaran/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/uttaran/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/uttaran/anaconda3/lib/python3.6/site-packages/tensorflow/python/estimator/inputs/queues/feeding_queue_runner.py", line 94, in _run
    sess.run(enqueue_op, feed_dict=feed_dict)
  File "/home/uttaran/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 889, in run
    run_metadata_ptr)
  File "/home/uttaran/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1047, in _run
    raise RuntimeError('Attempted to use a closed Session.')
RuntimeError: Attempted to use a closed Session.




%%%%%%% Train accuracy %%%%%%%%%%%%  {'accuracy': 0.7193313, 'loss': 1.0923642, 'global_step': 40000}
2 1 7 4 10
ERROR:tensorflow:Exception in QueueRunner: Attempted to use a closed Session.


Exception in thread Thread-1846:
Traceback (most recent call last):
  File "/home/uttaran/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/uttaran/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/uttaran/anaconda3/lib/python3.6/site-packages/tensorflow/python/estimator/inputs/queues/feeding_queue_runner.py", line 94, in _run
    sess.run(enqueue_op, feed_dict=feed_dict)
  File "/home/uttaran/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 889, in run
    run_metadata_ptr)
  File "/home/uttaran/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1047, in _run
    raise RuntimeError('Attempted to use a closed Session.')
RuntimeError: Attempted to use a closed Session.




######### Test accuracy #############  {'accuracy': 0.71849, 'loss': 1.0745244, 'global_step': 40000}
C1


In [22]:
writef.close()